## Predict Deep Learning Model

Predict the sentiment analysis label, using a deep learning model

In [7]:
# Add project path to the PYTHONPATH

import os
import sys
from pathlib import Path

sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())

In [8]:
import pandas as pd
from pathlib import Path

In [9]:
model_weights_path = Path('../models/model_weights.h5').resolve()
dataset_path = Path(
    '../datasets/emotion_from_query/1149033514022649856-1148812793153499142_:anxious_face_with_sweat:.csv'
).resolve()
tokenizer_file = Path('../datasets/sentiment140/tokenizer.pickle').resolve()

In [10]:
with tokenizer_file.open('rb') as file:
    tokenizer = pickle.load(file)

In [11]:
from sentiment_analysis.models.lstm_model import lstm_model

In [13]:
input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
model = lstm_model(100, input_dim, embedding_dim=200)
model.load_weights(model_weights_path.as_posix())

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
dataset = pd.read_csv(dataset_path)
predict_sequences = [text.split() for text in dataset.text]
list_tokenized_predict = tokenizer.texts_to_sequences(predict_sequences)
x_predict = pad_sequences(list_tokenized_predict, maxlen=100)

In [20]:
model.predict(x_predict)

array([[0.30173847, 0.6982615 ],
       [0.7938798 , 0.20612016],
       [0.06779335, 0.9322067 ],
       ...,
       [0.38065973, 0.6193403 ],
       [0.0894654 , 0.9105346 ],
       [0.39862603, 0.601374  ]], dtype=float32)

In [ ]:
dataset.text.iloc[1]
dataset.text.iloc[0]